In [143]:
import yaml
import glob
from shortid import ShortId
from rdflib import plugin, ConjunctiveGraph, Graph, URIRef, Literal, BNode
from rdflib.store import Store
from rdflib.namespace import RDF, RDFS, XSD

ns = 'http://envri.eu/ns/'
sid = ShortId()
store = plugin.get('IOMemory', Store)()

vocab = dict()
vocab['Bag'] = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Bag')
vocab['li'] = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#li')
vocab['ResearchInfrastructure'] = URIRef('http://www.oil-e.net/ontology/envri-rm.owl#ResearchInfrastructure')
vocab['Repository'] = URIRef('http://envri.eu/ns/Repository')
vocab['data_repository'] = URIRef('http://envri.eu/ns/DataRepository')
vocab['data repository'] = URIRef('http://envri.eu/ns/DataRepository')
vocab['metadata_repository'] = URIRef('http://envri.eu/ns/MetadataRepository')
vocab['metadata repository'] = URIRef('http://envri.eu/ns/MetadataRepository')
vocab['metadata_data_repository'] = URIRef('http://envri.eu/ns/MetadataRepository')
vocab['metadata data repository'] = URIRef('http://envri.eu/ns/MetadataRepository')
vocab['site_repository'] = URIRef('http://envri.eu/ns/SiteRepository')
vocab['taxonomic_repository'] = URIRef('http://envri.eu/ns/TaxonomicRepository')
vocab['Data'] = URIRef('http://envri.eu/ns/Data')
vocab['text_data'] = URIRef('http://envri.eu/ns/TextData')
vocab['images'] = URIRef('http://envri.eu/ns/Images')
vocab['tabular_data'] = URIRef('http://envri.eu/ns/TabularData')
vocab['binary_data'] = URIRef('http://envri.eu/ns/BinaryData')
vocab['binary'] = URIRef('http://envri.eu/ns/BinaryData')
vocab['structured_data'] = URIRef('http://envri.eu/ns/StructuredData')
vocab['PreferredFormat'] = URIRef('http://envri.eu/ns/PreferredFormat')
vocab['Metadata'] = URIRef('http://envri.eu/ns/Metadata')
vocab['codelist'] = URIRef('http://envri.eu/ns/Codelist')
vocab['thesaurus'] = URIRef('http://envri.eu/ns/Thesaurus')

vocab['Identifier'] = URIRef('http://envri.eu/ns/Identifier')
vocab['LocalIdentifier'] = URIRef('http://envri.eu/ns/LocalIdentifier')
vocab['local_ID'] = URIRef('http://envri.eu/ns/LocalIdentifier')
vocab['PersistentIdentifier'] = URIRef('http://envri.eu/ns/PersistentIdentifier')
vocab['PID'] = URIRef('http://envri.eu/ns/PersistentIdentifier')
vocab['PIDs'] = URIRef('http://envri.eu/ns/PersistentIdentifiers')
vocab['UUID'] = URIRef('http://envri.eu/ns/UniversallyUniqueIdentifier')
vocab['DigitalObjectIdentifier'] = URIRef('http://envri.eu/ns/DigitalObjectIdentifier')
vocab['MDA_ID'] = URIRef('http://envri.eu/ns/MDA_ID')
vocab['DOI'] = URIRef('http://envri.eu/ns/DigitalObjectIdentifier')
vocab['AccessMechanism'] = URIRef('http://envri.eu/ns/AccessMechanism')

vocab['date'] = URIRef('http://purl.org/dc/terms/date')
vocab['version'] = URIRef('http://purl.org/dc/terms/hasVersion')
vocab['creator'] = URIRef('http://purl.org/dc/terms/creator')
vocab['name'] = URIRef('http://xmlns.com/foaf/0.1/name')
vocab['email'] = URIRef('http://xmlns.com/foaf/0.1/mbox')
vocab['label'] = URIRef('http://www.w3.org/2000/01/rdf-schema#label')

vocab['acronym'] = URIRef('http://envri.eu/ns/acronym')
vocab['url'] = URIRef('http://envri.eu/ns/url')
vocab['hasDataManagementPlans'] = URIRef('http://envri.eu/ns/hasDataManagementPlans')
vocab['usesSpecificDataManagementPlanTools'] = URIRef('http://envri.eu/ns/usesSpecificDataManagementPlanTools')
vocab['appliedDataPublishingSteps'] = URIRef('http://envri.eu/ns/appliedDataPublishingSteps')
vocab['hasRepository'] = URIRef('http://envri.eu/ns/hasRepository')
vocab['usesSystem'] = URIRef('http://envri.eu/ns/usesSystem')
vocab['usesIdentifier'] = URIRef('http://envri.eu/ns/usesIdentifier')
vocab['usesProvider'] = URIRef('http://envri.eu/ns/usesProvider')
vocab['isAssigned'] = URIRef('http://envri.eu/ns/isAssigned')
vocab['hasCertificationMethod'] = URIRef('http://envri.eu/ns/hasCertificationMethod')
vocab['hasPolicy'] = URIRef('http://envri.eu/ns/hasPolicy')
vocab['inRegistry'] = URIRef('http://envri.eu/ns/inRegistry')
vocab['hasPersistentPolicy'] = URIRef('http://envri.eu/ns/hasPersistentPolicy')
vocab['hasPersistencyGuaranty'] = URIRef('http://envri.eu/ns/hasPersistencyGuaranty')
vocab['hasAccessMechanisms'] = URIRef('http://envri.eu/ns/hasAccessMechanisms')
vocab['hasAuthenticationMethod'] = URIRef('http://envri.eu/ns/hasAuthenticationMethod')
vocab['hasAccessProtocolUrl'] = URIRef('http://envri.eu/ns/hasAccessProtocolUrl')
vocab['protocolIsOpen'] = URIRef('http://envri.eu/ns/protocolIsOpen')
vocab['protocolIsRoyaltyFree'] = URIRef('http://envri.eu/ns/protocolIsRoyaltyFree')
vocab['maintainsOwnUserDatabase'] = URIRef('http://envri.eu/ns/maintainsOwnUserDatabase')
vocab['usesORCIDinAAI'] = URIRef('http://envri.eu/ns/usesORCIDinAAI')
vocab['supportsMajorAccessTechnology'] = URIRef('http://envri.eu/ns/supportsMajorAccessTechnology')
vocab['usesAuthorisationTechnique'] = URIRef('http://envri.eu/ns/usesAuthorisationTechnique')
vocab['usesAuthorisationFor'] = URIRef('http://envri.eu/ns/usesAuthorisationFor')
vocab['authorisationRequired'] = URIRef('http://envri.eu/ns/authorisationRequired')
vocab['hasAccessConcentProcessDescriptionUri'] = URIRef('http://envri.eu/ns/hasAccessConcentProcessDescriptionUri')
vocab['usesDataLicenses'] = URIRef('http://envri.eu/ns/usesDataLicenses')
vocab['dataLicenseIri'] = URIRef('http://envri.eu/ns/dataLicenseIri')
vocab['openAccessMetadata'] = URIRef('http://envri.eu/ns/openAccessMetadata')
vocab['hasData'] = URIRef('http://envri.eu/ns/hasData')
vocab['hasMetadata'] = URIRef('http://envri.eu/ns/hasMetadata')
vocab['hasPreferredFormat'] = URIRef('http://envri.eu/ns/hasPreferredFormat')
vocab['hasFormatName'] = URIRef('http://envri.eu/ns/hasFormatName')
vocab['hasDataHeaderMetadataTypes'] = URIRef('http://envri.eu/ns/hasDataHeaderMetadataTypes')
vocab['includedMetadata'] = URIRef('http://envri.eu/ns/includedMetadata')
vocab['hasMetadataHeaders'] = URIRef('http://envri.eu/ns/hasMetadataHeaders')
vocab['metadataTypes'] = URIRef('http://envri.eu/ns/metadataTypes')
vocab['dataSchemaIsRegistered'] = URIRef('http://envri.eu/ns/dataSchemaIsRegistered')
vocab['searchOnData'] = URIRef('http://envri.eu/ns/searchOnData')
vocab['hasSearchEngineUrl'] = URIRef('http://envri.eu/ns/hasSearchEngineUrl')
vocab['hasSchema'] = URIRef('http://envri.eu/ns/hasSchema')
vocab['hasIri'] = URIRef('http://envri.eu/ns/hasIri')
vocab['hasName'] = URIRef('http://envri.eu/ns/hasName')
vocab['includesProvenanceFields'] = URIRef('http://envri.eu/ns/includesProvenanceFields')
vocab['categoriesAreDefinedInRegistries'] = URIRef('http://envri.eu/ns/categoriesAreDefinedInRegistries')
vocab['persistentIdentifiersAreIncluded'] = URIRef('http://envri.eu/ns/persistentIdentifiersAreIncluded')
vocab['hasPrimaryStorageFormat'] = URIRef('http://envri.eu/ns/hasPrimaryStorageFormat')
vocab['hasMetadataLongevityPlan'] = URIRef('http://envri.eu/ns/hasMetadataLongevityPlan')
vocab['hasFormat'] = URIRef('http://envri.eu/ns/hasFormat')
vocab['supportedExportFormats'] = URIRef('http://envri.eu/ns/supportedExportFormats')
vocab['hasHarvestingMethod'] = URIRef('http://envri.eu/ns/hasHarvestingMethod')
vocab['hasLocalSearchEngine'] = URIRef('http://envri.eu/ns/hasLocalSearchEngine')
vocab['supportsExternalSearchEngineType'] = URIRef('http://envri.eu/ns/supportsExternalSearchEngineType')
vocab['includesAccessPolicyStatements'] = URIRef('http://envri.eu/ns/includesAccessPolicyStatements')
vocab['isMachineActionable'] = URIRef('http://envri.eu/ns/isMachineActionable')
vocab['hasVocabularies'] = URIRef('http://envri.eu/ns/hasVocabularies')
vocab['hasTopic'] = URIRef('http://envri.eu/ns/hasTopic')
vocab['hasSpecificationLanguage'] = URIRef('http://envri.eu/ns/hasSpecificationLanguage')
vocab['hasDataProcessing'] = URIRef('http://envri.eu/ns/hasDataProcessing')
vocab['specialDataProcessingStepsApplied'] = URIRef('http://envri.eu/ns/specialDataProcessingStepsApplied')
vocab['workflowFrameworksApplied'] = URIRef('http://envri.eu/ns/workflowFrameworksApplied')
vocab['distributedWorkflowsToolsUsed'] = URIRef('http://envri.eu/ns/distributedWorkflowsToolsUsed')
vocab['otherAnalysisServicesOffered'] = URIRef('http://envri.eu/ns/otherAnalysisServicesOffered')
vocab['dataProductsOffered'] = URIRef('http://envri.eu/ns/dataProductsOffered')
vocab['fairness'] = URIRef('http://envri.eu/ns/fairness')
vocab['dataFindability'] = URIRef('http://envri.eu/ns/dataFindability')
vocab['dataAccessibility'] = URIRef('http://envri.eu/ns/dataAccessibility')
vocab['dataInteroperability'] = URIRef('http://envri.eu/ns/dataInteroperability')
vocab['dataReusability'] = URIRef('http://envri.eu/ns/dataReusability')
vocab['dataIsFindable'] = URIRef('http://envri.eu/ns/dataIsFindable')
vocab['dataIsAccessible'] = URIRef('http://envri.eu/ns/dataIsAccessible')
vocab['dataIsInteroperable'] = URIRef('http://envri.eu/ns/dataIsInteroperable')
vocab['dataIsReusable'] = URIRef('http://envri.eu/ns/dataIsReusable')
vocab['gaps'] = URIRef('http://envri.eu/ns/gaps')
vocab['hasDataset'] = URIRef('http://envri.eu/ns/hasDataset')
vocab['hasDiscoveryPortal'] = URIRef('http://envri.eu/ns/hasDiscoveryPortal')
vocab['hasMachineReadableDatasetMetadata'] = URIRef('http://envri.eu/ns/hasMachineReadableDatasetMetadata')
vocab['hasLinksetDescribingResource'] = URIRef('http://envri.eu/ns/hasLinksetDescribingResource')
vocab['hasSearchQuery'] = URIRef('http://envri.eu/ns/hasSearchQuery')
vocab['hasCitationalProvenanceVocabulary'] = URIRef('http://envri.eu/ns/hasCitationalProvenanceVocabulary')
vocab['hasContextualProvenanceVocabulary'] = URIRef('http://envri.eu/ns/hasContextualProvenanceVocabulary')
vocab['hasComplianceCertification'] = URIRef('http://envri.eu/ns/hasComplianceCertification')

vocab['domain specific'] = URIRef('http://envri.eu/ns/DomainSpecific')
vocab['ISO_10139'] = URIRef('http://envri.eu/ns/ISO_19139')
vocab['ISO 19139'] = URIRef('http://envri.eu/ns/ISO_19139')
vocab['ISO_19115'] = URIRef('http://envri.eu/ns/ISO_19115')
vocab['ISO 19115'] = URIRef('http://envri.eu/ns/ISO_19115')
vocab['ISO19139'] = URIRef('http://envri.eu/ns/ISO_19139')
vocab['ISO TC211'] = URIRef('http://envri.eu/ns/ISO_TC211')
vocab['INSPIRE_EF'] = URIRef('http://envri.eu/ns/INSPIRE_EF')
vocab['INSPIRE EF'] = URIRef('http://envri.eu/ns/INSPIRE_EF')
vocab['INSPIRE'] = URIRef('http://envri.eu/ns/INSPIRE')
vocab['EML2.0'] = URIRef('http://envri.eu/ns/EML2.0')
vocab['EML 2.0'] = URIRef('http://envri.eu/ns/EML2.0')
vocab['SensorML'] = URIRef('http://envri.eu/ns/SensorML')
vocab['SDN community profile'] = URIRef('http://envri.eu/ns/SeaDataNetCommunityProfile')
vocab['local'] = URIRef('http://envri.eu/ns/Local')
vocab['local_license'] = URIRef('http://envri.eu/ns/LocalLicense')
vocab['download_data'] = URIRef('http://envri.eu/ns/DataDownload')
vocab['download data'] = URIRef('http://envri.eu/ns/DataDownload')
vocab['upload_data'] = URIRef('http://envri.eu/ns/DataUpload')
vocab['upload data'] = URIRef('http://envri.eu/ns/DataUpload')
vocab['advanced_editing'] = URIRef('http://envri.eu/ns/AdvancedEditing')
vocab['Marine.id'] = URIRef('http://envri.eu/ns/Marine.id')
vocab['OAUTH'] = URIRef('http://envri.eu/ns/OAUTH')
vocab['CAS'] = URIRef('http://envri.eu/ns/CAS')
vocab['Zenodo'] = URIRef('http://envri.eu/ns/Zenodo')
vocab['planned'] = URIRef('http://envri.eu/ns/Planned')
vocab['Sextant-Si'] = URIRef('http://envri.eu/ns/Sextant-Si')
vocab['Sextant'] = URIRef('http://envri.eu/ns/Sextant')
vocab['52_NORTH_SOS'] = URIRef('http://envri.eu/ns/52NORTH_SOS')
vocab['52NORTH SOS'] = URIRef('http://envri.eu/ns/52NORTH_SOS')
vocab['SEANOE'] = URIRef('http://envri.eu/ns/SEANOE')
vocab['MS_SQL'] = URIRef('http://envri.eu/ns/MS_SQL')
vocab['Handle'] = URIRef('http://envri.eu/ns/Handle')
vocab['local_system'] = URIRef('http://envri.eu/ns/LocalSystem')
vocab['local_service'] = URIRef('http://envri.eu/ns/LocalService')
vocab['local_LDAP'] = URIRef('http://envri.eu/ns/LocalLDAP')
vocab['local LDAP'] = URIRef('http://envri.eu/ns/LocalLDAP')
vocab['DEIMS-SDR'] = URIRef('http://envri.eu/ns/DEIMS-SDR')
vocab['DataCite'] = URIRef('http://envri.eu/ns/DataCite')
vocab['re3data'] = URIRef('http://envri.eu/ns/re3data')
vocab['ASCII'] = URIRef('http://envri.eu/ns/ASCII')
vocab['xls'] = URIRef('http://envri.eu/ns/xls')
vocab['NetCDF'] = URIRef('http://envri.eu/ns/NetCDF')
vocab['ODV'] = URIRef('http://envri.eu/ns/OceanDataView')
vocab['CSV'] = URIRef('http://envri.eu/ns/CommaSeparatedValues')
vocab['XML'] = URIRef('http://envri.eu/ns/ExtensibleMarkupLanguage')
vocab['HTML'] = URIRef('http://envri.eu/ns/HyperTextMarkupLanguage')
vocab['PDF'] = URIRef('http://envri.eu/ns/PortableDocumentFormat')
vocab['text'] = URIRef('http://envri.eu/ns/Text')
vocab['text only'] = URIRef('http://envri.eu/ns/TextOnly')
vocab['OAI-PMH'] = URIRef('http://envri.eu/ns/OAI-PMH')
vocab['OAI PMH'] = URIRef('http://envri.eu/ns/OAI-PMH')
vocab['OGC_CSW'] = URIRef('http://envri.eu/ns/OGC_CSW')
vocab['OGC CSW'] = URIRef('http://envri.eu/ns/OGC_CSW')
vocab['CSW'] = URIRef('http://envri.eu/ns/OGC_CSW')
vocab['CSW_endpoint'] = URIRef('http://envri.eu/ns/OGC_CSW')
vocab['GEOPortal'] = URIRef('http://envri.eu/ns/GEOPortal')
vocab['CDI system'] = URIRef('http://envri.eu/ns/CDISystem')
vocab['parameters_measured'] = URIRef('http://envri.eu/ns/ParametersMeasured')
vocab['parameters measured'] = URIRef('http://envri.eu/ns/ParametersMeasured')
vocab['access content'] = URIRef('http://envri.eu/ns/AccessContent')
vocab['data centre'] = URIRef('http://envri.eu/ns/DataCentre')
vocab['date_time'] = URIRef('http://envri.eu/ns/DateTime')
vocab['time'] = URIRef('http://envri.eu/ns/Time')
vocab['location'] = URIRef('http://envri.eu/ns/Location')
vocab['temperature'] = URIRef('http://envri.eu/ns/Temperature')
vocab['vegetation'] = URIRef('http://envri.eu/ns/Vegetation')
vocab['salinity'] = URIRef('http://envri.eu/ns/Salinity')
vocab['depth'] = URIRef('http://envri.eu/ns/Depth')
vocab['climatologies'] = URIRef('http://envri.eu/ns/Climatologies')
vocab['deposition'] = URIRef('http://envri.eu/ns/Deposition')
vocab['mesoclimate'] = URIRef('http://envri.eu/ns/Mesoclimate')
vocab['partially'] = URIRef('http://envri.eu/ns/Partially')
vocab['none'] = URIRef('http://envri.eu/ns/None')
vocab[None] = URIRef('http://envri.eu/ns/NULL')
vocab['NULL'] = URIRef('http://envri.eu/ns/NULL')
vocab['VOID'] = URIRef('http://envri.eu/ns/VOID')
vocab['ODV aggregation'] = URIRef('http://envri.eu/ns/ODVAggregation')
vocab['QA QC'] = URIRef('http://envri.eu/ns/QA_QC')
vocab['DIVA'] = URIRef('http://envri.eu/ns/DIVA')
vocab['creative_commons'] = URIRef('http://envri.eu/ns/CreativeCommons')
vocab['interpolations'] = URIRef('http://envri.eu/ns/Interpolations')
vocab['uptake in central resources'] = URIRef('http://envri.eu/ns/UptakeInCentralResources')
vocab['problem with FTP access'] = URIRef('http://envri.eu/ns/FTPAccessProblems')
vocab['provenance information'] = URIRef('http://envri.eu/ns/ProvenanceInformation')
vocab['quality description'] = URIRef('http://envri.eu/ns/QualityDescription')
vocab['machine accessibility'] = URIRef('http://envri.eu/ns/MachineAccessibility')
vocab['usage license'] = URIRef('http://envri.eu/ns/UsageLicense')
vocab['distribution link'] = URIRef('http://envri.eu/ns/DistributionLink')
vocab['format checking'] = URIRef('http://envri.eu/ns/FormatChecking')
vocab['data quality control'] = URIRef('http://envri.eu/ns/QA_QC')

def _l(g, d, n, k, t):
    if isinstance(k, list):
        p = vocab[k[0]]
        o = d[k[1]]
    else:
        p = vocab[k]
        o = d[k]

    if o == 'VOID':
        return
    
    g.add((n, p, Literal(o, datatype=t)))
    
def _b(g, n, k, b):
    g.add((n, vocab[k], b))
    
def _r(g, d, n, k):
    if isinstance(k, list):
        p = vocab[k[0]]
        if d is None:
            o = vocab['NULL']
        elif d == 'VOID':
            o = vocab['VOID']
        elif d == 'none':
            o = vocab['none']
        elif d == 'partially':
            o = vocab['partially']
        else: 
            o = d[k[1]]
    else:
        p = vocab[k]
        o = d[k]
        
    if o == 'VOID':
        return
    
    if o is None:
        g.add((n, p, URIRef(vocab[o])))
        return
    
    if o.find('http') > -1 or o.find('www') > -1 or o.find('@') > -1:
        g.add((n, p, URIRef(o)))
        return
        
    g.add((n, p, URIRef(vocab[o])))
    
def _t(g, d, n, k):
    if k in d:
        g.add((n, RDF.type, vocab[d[k]]))
    else:
        g.add((n, RDF.type, vocab[k]))
    
def _c(g, d, n1, n2, k):
    _b(g, n1, k, n2)
    _t(g, d, n2, 'Bag')
    for e in d:
        _li(g, n2, e)
        
def _li(g, n, e):
    g.add((n, vocab['li'], URIRef(vocab[e])))
    
def process(d):
    gid = URIRef('{}G{}'.format(ns, sid.generate()))
    g = Graph(store, gid)
    process_survey(g, d['survey'])
    process_infrastructure(g, d['infrastructure'])
    
def process_survey(g, d):
    n = BNode()
    _l(g, d, g.identifier, 'date', XSD.date)
    _l(g, d, g.identifier, 'version', XSD.string)
    process_creator(g, d['creator'], g.identifier)
    
def process_creator(g, d, n):
    n1 = BNode()
    _b(g, n, 'creator', n1)
    _l(g, d, n1, 'name', XSD.string)
    _r(g, d, n1, 'email')
    
def process_infrastructure(g, d):
    n = BNode()
    _t(g, d, n, 'ResearchInfrastructure')
    _l(g, d, n, 'acronym', XSD.string)
    _l(g, d, n, ['label', 'name'], XSD.string)
    _r(g, d, n, ['url', 'recognized authority IRI'])
    process_infrastructure_datamanagementplans(g, d['data management plans'], n)
    for repository in d['repositories']:
        process_repository(g, repository, n)
    
def process_infrastructure_datamanagementplans(g, d, n):
    if (handle_special_cases(g, d, n, 'hasDataManagementPlans')):
        return
    n1 = BNode()
    _b(g, n, 'hasDataManagementPlans', n1)
    _l(g, d, n1, ['usesSpecificDataManagementPlanTools', 'specific DMP tools used'], XSD.bool)
    _l(g, d, n1, ['appliedDataPublishingSteps', 'data publishing steps applied'], XSD.string)
        
def process_repository(g, d, n):
    if (handle_special_cases(g, d, n, 'hasRepository')):
        return
    n1 = BNode()
    _b(g, n, 'hasRepository', n1)
    _t(g, d, n1, 'Repository')
    _l(g, d, n1, ['label', 'name'], XSD.string)  
    _r(g, d, n1, ['url', 'IRI'])
    _t(g, d, n1, 'kind')
    _r(g, d, n1, ['usesSystem', 'software'])
    process_repository_identifier(g, d['identifier'], n1)
    _l(g, d, n1, ['hasCertificationMethod', 'certification method'], XSD.string)
    _l(g, d, n1, ['hasPolicy', 'policy'], XSD.string)
    _r(g, d, n1, ['inRegistry', 'registry'])
    _l(g, d, n1, ['hasPersistencyGuaranty', 'persistency-guaranty'], XSD.string)
    process_repository_access(g, d['access mechanisms'], n1)
    process_repository_data(g, d['data'], n1)
    process_repository_metadata(g, d['metadata'], n1)
    process_repository_vocabularies(g, d['vocabularies'], n1)
    process_repository_dataprocessing(g, d['data processing'], n1)
    process_repository_fairness(g, d['fairness'], n1)
    process_repository_testfairness(g, d['test fairness'], n1)
        
        
def process_repository_identifier(g, d, n):
    if (handle_special_cases(g, d, n, 'usesIdentifier')):
        return
    n1 = BNode()
    _b(g, n, 'usesIdentifier', n1)
    _t(g, d, n1, 'Identifier')
    _t(g, d, n1, 'kind')
    _r(g, d, n1, ['usesSystem', 'system'])
    _l(g, d, n1, ['isAssigned', 'assigned'], XSD.string)
    _r(g, d, n1, ['usesProvider', 'provider'])
    
    
def process_repository_access(g, d, n):
    if (handle_special_cases(g, d, n, 'hasAccessMechanisms')):
        return
    n1 = BNode()
    _b(g, n, 'hasAccessMechanisms', n1)
    _t(g, d, n1, 'AccessMechanism')
    _l(g, d, n1, ['hasAuthenticationMethod', 'authentication method'], XSD.string)
    _r(g, d, n1, ['hasAccessProtocolUrl', 'access protocol URL'])
    _l(g, d, n1, ['protocolIsOpen', 'protocol open'], XSD.bool)
    _l(g, d, n1, ['protocolIsRoyaltyFree', 'protocol royalty free'], XSD.bool)
    _l(g, d, n1, ['maintainsOwnUserDatabase', 'own user database maintained'], XSD.bool)
    _l(g, d, n1, ['usesORCIDinAAI', 'ORCID used in AAI'], XSD.bool)
    _r(g, d, n1, ['supportsMajorAccessTechnology', 'major access technology supported'])
    _r(g, d, n1, ['usesAuthorisationTechnique', 'authorisation technique'])
    process_authorisation_needed_for(g, d['authorisation needed for'], n1)
    _l(g, d, n1, ['authorisationRequired', 'authorization for accessing content needed'], XSD.bool)
    _r(g, d, n1, ['hasAccessConcentProcessDescriptionUri', 'access content process description IRI'])
#    _r(g, d, n1, ['usesDataLicenses', 'data licenses in use'])
    _r(g, d, n1, ['dataLicenseIri', 'data license IRI'])
    _l(g, d, n1, ['openAccessMetadata', 'metadata openly available'], XSD.bool)
    
    
def process_authorisation_needed_for(g, d, n):
    if (handle_special_cases(g, d, n, 'usesAuthorisationFor')):
        return
    n1 = BNode()
    _b(g, n, 'usesAuthorisationFor', n1)
    _t(g, d, n1, 'Bag')
    for e in d:
        _li(g, n1, e)
    
def process_repository_data(g, d, n):
    if (handle_special_cases(g, d, n, 'hasData')):
        return
    for e1 in d:
        n1 = BNode()
        _b(g, n, 'hasData', n1)
        _t(g, e1, n1, 'Data')
        _t(g, e1, n1, 'type name')
        _l(g, e1, n, ['dataSchemaIsRegistered', 'registered data schema'], XSD.bool)
        _l(g, e1, n, ['searchOnData', 'search on data'], XSD.bool)
        _r(g, e1, n, ['hasSearchEngineUrl', 'search engine URL'])
        for e2 in e1['preferred formats']:
            n2 = BNode()
            _b(g, n1, 'hasPreferredFormat', n2)
            _t(g, e2, n2, 'PreferredFormat')
            _r(g, e2, n2, ['hasFormatName', 'format name'])
            e3 = e2['metadata types in data headers']
            if e3 is None:
                _r(g, e3, n1, ['hasDataHeaderMetadataTypes', 'NULL'])
            elif e3 == 'none':
                _r(g, e3, n1, ['hasDataHeaderMetadataTypes', 'None'])
            else:
                n3 = BNode()
                _b(g, n2, 'hasDataHeaderMetadataTypes', n3)
                _t(g, e3, n3, 'Bag')
                for e in e3:
                    _li(g, n3, e)
                

def process_repository_metadata(g, d, n):
    if handle_special_cases(g, d, n, 'hasMetadata'):
        return
    n1 = BNode()
    _b(g, n, 'hasMetadata', n1)
    process_repository_metadata_schema(g, d['schema'], n1)
    _l(g, d, n1, ['categoriesAreDefinedInRegistries', 'categories defined in registries'], XSD.bool)
    _l(g, d, n1, ['persistentIdentifiersAreIncluded', 'PIDs included'], XSD.bool)
    _r(g, d, n1, ['hasPrimaryStorageFormat', 'primary storage format'])
    _r(g, d, n1, ['hasMetadataLongevityPlan', 'metadata longevity plan URL'])
    _r(g, d, n1, ['hasFormat', 'format IRI'])
    n2 = BNode()
    _b(g, n1, 'supportedExportFormats', n2)
    _t(g, d, n2, 'Bag')
    for e in d['export format supported']:
        _li(g, n2, e)
    n3 = BNode()
    _b(g, n1, 'hasHarvestingMethod', n3)
    _t(g, d, n3, 'Bag')
    for e in d['exchange/harvesting method']:
        _li(g, n3, e)
    _r(g, d, n1, ['hasLocalSearchEngine', 'local search engine URL'])
    _r(g, d, n1, ['supportsExternalSearchEngineType', 'external search engine type supported'])
    _l(g, d, n1, ['includesAccessPolicyStatements', 'access policy statements included'], XSD.bool)
    _l(g, d, n1, ['isMachineActionable', 'machine actionable'], XSD.bool)


def process_repository_metadata_schema(g, d, n):
    if handle_special_cases(g, d, n, 'hasSchema'):
        return
    for e1 in d:
        n1 = BNode()
        _b(g, n, 'hasSchema', n1)
        _r(g, e1, n1, ['hasIri', 'IRI'])
        _r(g, e1, n1, ['hasName', 'name'])
        e2 = e1['provenance fields included']
        if e2 is None:
            _r(g, e2, n1, ['includesProvenanceFields', 'None'])
        elif e2 == 'partially':
            _r(g, e2, n1, ['includesProvenanceFields', 'partially'])
        else:
            n2 = BNode()
            _b(g, n1, 'includesProvenanceFields', n2)
            _t(g, e1, n2, 'Bag')
            for e2 in e2:
                _li(g, n2, e2)
    
    
def process_repository_vocabularies(g, d, n):
    if handle_special_cases(g, d, n, 'hasVocabularies'):
        return
    for e1 in d:
        n1 = BNode()
        _b(g, n, 'hasVocabularies', n1)
        _r(g, e1, n1, ['hasIri', 'IRI'])
        _t(g, e1, n1, 'type')
        _r(g, e1, n1, ['hasTopic', 'topic'])
        _l(g, e1, n1, ['hasName', 'name'], XSD.string)
        _r(g, e1, n1, ['hasSpecificationLanguage', 'specification language URL'])

def process_repository_dataprocessing(g, d, n):
    if handle_special_cases(g, d, n, 'hasDataProcessing'):
        return
    n1 = BNode()
    _b(g, n, 'hasDataProcessing', n1)
    process_repository_dataprocessing_special(g, d['special data processing steps applied'], n1)
    process_repository_dataprocessing_workflow(g, d['workflow frameworks applied'], n1)
    process_repository_dataprocessing_distributed(g, d['distributed workflows tools used'], n1)
    process_repository_dataprocessing_other(g, d['other analysis services offered'], n1)
    process_repository_dataprocessing_data(g, d['data products offered'], n1)


def process_repository_dataprocessing_special(g, d, n):
    if handle_special_cases(g, d, n, 'specialDataProcessingStepsApplied'):
        return
    n1 = BNode()
    _b(g, n, 'specialDataProcessingStepsApplied', n1)
    _t(g, d, n1, 'Bag')
    for e in d:
        _li(g, n1, e)
    

def process_repository_dataprocessing_workflow(g, d, n):
    if handle_special_cases(g, d, n, 'workflowFrameworksApplied'):
        return
    n1 = BNode()
    _b(g, n, 'workflowFrameworksApplied', n1)
    _t(g, d, n1, 'Bag')
    for e in d:
        _li(g, n1, e)

def process_repository_dataprocessing_distributed(g, d, n):
    if handle_special_cases(g, d, n, 'distributedWorkflowsToolsUsed'):
        return
    n1 = BNode()
    _b(g, n, 'distributedWorkflowsToolsUsed', n1)
    _t(g, d, n1, 'Bag')
    for e in d:
        _li(g, n1, e)

def process_repository_dataprocessing_other(g, d, n):
    if handle_special_cases(g, d, n, 'otherAnalysisServicesOffered'):
        return
    n1 = BNode()
    _b(g, n, 'otherAnalysisServicesOffered', n1)
    _t(g, d, n1, 'Bag')
    for e in d:
        _li(g, n1, e)

def process_repository_dataprocessing_data(g, d, n):
    if handle_special_cases(g, d, n, 'dataProductsOffered'):
        return
    n1 = BNode()
    _b(g, n, 'dataProductsOffered', n1)
    _t(g, d, n1, 'Bag')
    for e in d:
        _li(g, n1, e)
    
def process_repository_fairness(g, d, n):
    if handle_special_cases(g, d, n, 'fairness'):
        return
    n1 = BNode()
    _b(g, n, 'fairness', n1)
    process_repository_faireness_findability(g, d['data findability'], n1)
    process_repository_faireness_accessibility(g, d['data accessibility'], n1)
    process_repository_faireness_interoperability(g, d['data interoperability'], n1)
    process_repository_faireness_reusability(g, d['data re-usability'], n1)

def process_repository_faireness_findability(g, d, n):
    if handle_special_cases(g, d, n, 'dataFindability'):
        return
    n1 = BNode()
    _b(g, n, 'dataFindability', n1)
    _l(g, d, n1, ['dataIsFindable', 'data findable'], XSD.bool)
    process_repository_faireness_findability_gaps(g, d['gaps'], n1)

def process_repository_faireness_accessibility(g, d, n):
    if handle_special_cases(g, d, n, 'dataAccessibility'):
        return
    n1 = BNode()
    _b(g, n, 'dataAccessibility', n1)
    _l(g, d, n1, ['dataIsAccessible', 'data accessible'], XSD.bool)
    process_repository_faireness_accessibility_gaps(g, d['gaps'], n1)

def process_repository_faireness_interoperability(g, d, n):
    if handle_special_cases(g, d, n, 'dataInteroperability'):
        return
    n1 = BNode()
    _b(g, n, 'dataInteroperability', n1)
    _l(g, d, n1, ['dataIsInteroperable', 'data interoperable'], XSD.bool)
    process_repository_faireness_interoperability_gaps(g, d['gaps'], n1)

def process_repository_faireness_reusability(g, d, n):
    if handle_special_cases(g, d, n, 'dataReusability'):
        return
    n1 = BNode()
    _b(g, n, 'dataReusability', n1)
    _l(g, d, n1, ['dataIsReusable', 'data reusable'], XSD.bool)
    process_repository_faireness_reusability_gaps(g, d['gaps'], n1)
    
def process_repository_faireness_findability_gaps(g, d, n):
    if handle_special_cases(g, d, n, 'gaps'):
        return
    _c(g, d, n, BNode(), 'gaps')
    
def process_repository_faireness_accessibility_gaps(g, d, n):
    if handle_special_cases(g, d, n, 'gaps'):
        return
    _c(g, d, n, BNode(), 'gaps')

def process_repository_faireness_interoperability_gaps(g, d, n):
    if handle_special_cases(g, d, n, 'gaps'):
        return
    _c(g, d, n, BNode(), 'gaps')
    
def process_repository_faireness_reusability_gaps(g, d, n):
    if handle_special_cases(g, d, n, 'gaps'):
        return
    _c(g, d, n, BNode(), 'gaps')
    
def process_repository_testfairness(g, d, n):
    _r(g, d, n, ['hasDataset', 'URL/IRI of dataset'])
    _r(g, d, n, ['hasDiscoveryPortal', 'URL of discovery portal'])
    _r(g, d, n, ['hasMachineReadableDatasetMetadata', 'IRI of machine readable metadata of dataset'])
    _r(g, d, n, ['hasLinksetDescribingResource', 'URL to linkset describing resource'])
    _l(g, d, n, ['hasSearchQuery', 'Search query'], XSD.string)
    _r(g, d, n, ['hasCitationalProvenanceVocabulary', 'IRI for the vocabulary used to describe citational provenance'])
    _r(g, d, n, ['hasContextualProvenanceVocabulary', 'IRI for the vocabulary used to describe contextual provenance'])
    _r(g, d, n, ['hasComplianceCertification', 'IRI of compliance certification'])

def handle_special_cases(g, d, n, k):
    if d is None:
        _r(g, d, n, [k, 'NULL'])
        return True
    if d is 'NULL':
        _r(g, d, n, [k, 'NULL'])
        return True
    if d == 'VOID':
        _r(g, d, n, [k, 'VOID'])
        return True
    if d == 'none':
        _r(g, d, n, [k, 'none'])
        return True
    return False
    
for file in glob.glob('descriptions/*.yaml'):
    with open(file, 'r') as f:
        for document in yaml.load_all(f, Loader=yaml.FullLoader):
            process(document)
    
g = ConjunctiveGraph(store)
g.bind('envri', ns)
g.bind('dcterms', 'http://purl.org/dc/terms/')
g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.serialize(destination='data.trig', format='trig')

In [145]:
import io
import pandas as pd
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(io.StringIO(output.getvalue().decode('utf-8'))))

In [147]:
query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_url ?rep_label WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:url ?ri_url .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:Repository .
        ?rep rdfs:label ?rep_label .
    }
    FILTER (?date > "2019-03-15"^^xsd:date)
}
""")

,date,ri_acronym,ri_url,rep_label
0,2019-03-29,SDN,www.seadatanet.org,Data Products Catalogue
1,2019-03-29,SDN,www.seadatanet.org,SDN CDI
2,2019-03-19,eLTER,http://www.lter-europe.net/elter,EUDAT/FZJ B2SHARE
3,2019-03-19,eLTER,http://www.lter-europe.net/elter,DEIMS-SDR
4,2019-03-19,eLTER,http://www.lter-europe.net/elter,eLTER CDN
